In [112]:
import pandas as pd
import numpy as np
import LinearModels as LM
import matplotlib.pyplot as plt
import NonLinearModels_post as nlm

from scipy.stats import norm
from scipy.stats import genextreme

%load_ext autoreload
%autoreload 2

import clogit 
import estimation as est

import logit
import estimation2 as est2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pd.options.display.float_format = '{:.12f}'.format
path = "/Users/Lasse/Documents/Bachelor/Seb/MotorRegisterData-main/"

In [3]:
# CHOICE DATA
choice_data = pd.read_csv(path + 'choice_data.csv', delimiter=';', encoding = 'unicode_escape')
choice_data.rename(columns = {'No. of registrations': 'Registrations'}, inplace = True)
dummy_fuel = pd.get_dummies(choice_data['Fuel'])
choice_data = choice_data.join(dummy_fuel)
choice_data

Year                   Make-model-year-fuel        Make           Model  \
0     2006             toyota-avensis-2006-Benzin      toyota         avensis   
1     2006          volkswagen-passat-2006-Diesel  volkswagen          passat   
2     2006               renault-clio-2006-Benzin     renault            clio   
3     2006               suzuki-swift-2006-Benzin      suzuki           swift   
4     2006                 fiat-punto-2006-Benzin        fiat           punto   
...    ...                                    ...         ...             ...   
3377  2020         volkswagen-touareg-2020-Diesel  volkswagen         touareg   
3378  2020  volkswagen-golf-sportsvan-2020-Benzin  volkswagen  golf sportsvan   
3379  2020                  volkswagen-up-2020-El  volkswagen              up   
3380  2020         volkswagen-t-cross-2020-Benzin  volkswagen         t-cross   
3381  2020           volkswagen-t-roc-2020-Benzin  volkswagen           t-roc   

                            Variant Size segment Fuel-size segment  \
0                      1,8 STW MAN.        Small       BenzinSmall   
1                           2,0 TDI        Large       DieselLarge   
2                      1,2 8V 60 HK        Small       BenzinSmall   
3                               1,5        Small       BenzinSmall   
4                               1,2        Small       BenzinSmall   
...                             ...          ...               ...   
3377              3.0 TDI V6 286 HK        Large       DieselLarge   
3378           1.5 TSI EVO 150 DSG7        Large       BenzinLarge   
3379  ELEKTRO MOTOR 82 Aut. 1 trins        Small           ElSmall   
3380             1.0 TSI 115HK DSG7        Small       BenzinSmall   
3381             1.5 TSI 150HK DSG7        Large       BenzinLarge   

      Registrations         Shares   Prices (2015-DKK)  ...    Fuel  \
0              5428 0.002156418000 403349.299100000004  ...  Benzin   
1              5378 0.002136554000 378504.672900000005  ...  Diesel   
2              4518 0.001794896000 134344.626199999999  ...  Benzin   
3              4406 0.001750401000 163536.214999999997  ...  Benzin   
4              3743 0.001487007000 150287.383200000011  ...  Benzin   
...             ...            ...                 ...  ...     ...   
3377             35 0.000012800000 977674.081200000015  ...  Diesel   
3378            818 0.000299839000 303593.810400000017  ...  Benzin   
3379            370 0.000135624000 153689.555099999998  ...      El   
3380           2969 0.001088290000 263941.972899999993  ...  Benzin   
3381           3299 0.001209252000 254948.742700000003  ...  Benzin   

      Engine effect (kW)       Horsepower       Size (m3)  Footprint (m²)  \
0       108.000000000000 139.307692300000 12.573440000000  8.223261538000   
1       103.000000000000 128.333333300000 12.734904000000  8.657173611000   
2        43.000000000000  60.000000000000  8.872728000000  6.248400000000   
3        68.000000000000  94.000000000000  9.379500000000  6.253000000000   
4        48.000000000000  63.888888890000  9.483248000000  6.392844444000   
...                  ...              ...             ...             ...   
3377    210.000000000000 286.000000000000 16.619328000000  9.662400000000   
3378    110.000000000000 150.000000000000 12.411532000000 18.835600000000   
3379     61.000000000000  61.222222220000  8.855775289000  5.905600000000   
3380     85.000000000000 115.000000000000 11.429088000000 16.892100000000   
3381    110.000000000000 150.000000000000 12.086802000000 17.892900000000   

      Cost/km (DKK)  fuelPrice (DKK/unit)  Benzin  Diesel  El  
0    0.813669065000       11.310000000000       1       0   0  
1    0.681879195000       10.160000000000       0       1   0  
2    0.677245509000       11.310000000000       1       0   0  
3    0.734415584000       11.310000000000       1       0   0  
4    0.689634146000       11.310000000000       1       0   0  
...             ...           

In [4]:
len(choice_data[ choice_data['Year'] == 2020 ].index)

214

In [5]:
year = 2006
for x in range(2006,2021):
    obs_count = choice_data.loc[(choice_data['Year'] == year)]
    print(len(obs_count))
    year += 1

186
210
215
198
217
225
232
242
248
246
246
239
234
230
214


In [6]:
year = 2006
for x in range(2006,2021):
     while len(choice_data.loc[choice_data['Year'] == year]) > len(choice_data.loc[choice_data['Year'] == 2006]):
          min_val = choice_data.loc[(choice_data['Year'] == year) & (choice_data['Fuel'] != 'El'), 'Registrations'].idxmin()
          choice_data.drop(index=min_val,inplace=True)
     year += 1
choice_data.to_csv(path + 'clogitchoice_data.csv')

In [7]:
year = 2006
for x in range(2006,2021):
    obs_count = choice_data.loc[(choice_data['Year'] == year)]
    print(len(obs_count))
    year += 1

186
186
186
186
186
186
186
186
186
186
186
186
186
186
186


In [117]:
total_reg = choice_data['Registrations'].sum(axis=0)
choice_data['MarketShare'] = choice_data['Registrations'] / total_reg 
choice_data.head()

Year           Make-model-year-fuel        Make    Model       Variant  \
0  2006     toyota-avensis-2006-Benzin      toyota  avensis  1,8 STW MAN.   
1  2006  volkswagen-passat-2006-Diesel  volkswagen   passat       2,0 TDI   
2  2006       renault-clio-2006-Benzin     renault     clio  1,2 8V 60 HK   
3  2006       suzuki-swift-2006-Benzin      suzuki    swift           1,5   
4  2006         fiat-punto-2006-Benzin        fiat    punto           1,2   

  Size segment Fuel-size segment  Registrations         Shares  \
0        Small       BenzinSmall           5428 0.002156418000   
1        Large       DieselLarge           5378 0.002136554000   
2        Small       BenzinSmall           4518 0.001794896000   
3        Small       BenzinSmall           4406 0.001750401000   
4        Small       BenzinSmall           3743 0.001487007000   

    Prices (2015-DKK)  ...       Size (m3)  Footprint (m²)  Cost/km (DKK)  \
0 403349.299100000004  ... 12.573440000000  8.223261538000 0.813669065000   
1 378504.672900000005  ... 12.734904000000  8.657173611000 0.681879195000   
2 134344.626199999999  ...  8.872728000000  6.248400000000 0.677245509000   
3 163536.214999999997  ...  9.379500000000  6.253000000000 0.734415584000   
4 150287.383200000011  ...  9.483248000000  6.392844444000 0.689634146000   

   fuelPrice (DKK/unit)  Benzin  Diesel  El    market_year  \
0       11.310000000000       1       0   0 0.033475590202   
1       10.160000000000       0       1   0 0.033167229938   
2       11.310000000000       1       0   0 0.027863433407   
3       11.310000000000       1       0   0 0.027172706416   
4       11.310000000000       1       0   0 0.023083849323   

                             key    MarketShare  
0     toyota-avensis-2006-Benzin 0.001802670663  
1  volkswagen-passat-2006-Diesel 0.001786065369  
2       renault-clio-2006-Benzin 0.001500454321  
3       suzuki-swift-2006-Benzin 0.001463258463  
4         fiat-punto-2006-Benzin 0.001243072272  

[5 rows x 25 columns]

In [8]:
year = 2006
choice_data['market_year'] = None
for x in range(2006,2021):
    total_year = choice_data.loc[(choice_data['Year'] == year), 'Registrations'].sum(axis=0)
    share_year = ((choice_data.loc[(choice_data['Year'] == year), 'Registrations']) / total_year)
    new_choice_data = pd.DataFrame({'market_year': share_year})
    choice_data.update(new_choice_data)
    year += 1

In [9]:
choice_data['market_year']

0      0.033475590202
1      0.033167229938
2      0.027863433407
3      0.027172706416
4      0.023083849323
            ...      
3353   0.000386870922
3378   0.003859273343
3379   0.001745637087
3380   0.014007558137
3381   0.015564477701
Name: market_year, Length: 2790, dtype: object

In [118]:
# Byt kolonner
choice_data['key'] = choice_data['Model'].str.replace('jaguar ', '-')
choice_data['key'] = choice_data['Make'] + '-' + choice_data['Model'] + '-' + choice_data['Year'].astype(str) + '-' + choice_data['Fuel']
choice_data['key'] = choice_data['key'].str.replace(' ', '-')
choice_data_subset = choice_data[['Fuel', 'Benzin', 'Diesel', 'El', 'Make-model-year-fuel', 'Year', 'Make', 'market_year', 'MarketShare', 'Weight (kg)', 'Engine effect (kW)', 'Prices (2015-DKK)', 'Cost/km (DKK)', 'Size (m3)', 'Horsepower']]
choice_data_subset.rename(columns={ choice_data_subset.columns[0]: "Fuel" })
choice_data_subset.insert(0, "ID", range(0, 0 + len(choice_data_subset)))
#choice_data_subset.insert(-1, "y", np.select(conditions, choices))
## Lav en J for fuel: {0,1,2}
## En y for fuel, fx hvis benzin valgt så y = 0, diesel = 1 og el = 2

In [119]:
conditions = [
    (choice_data_subset['Fuel'] == 'El'),
    (choice_data_subset['Fuel'] == 'Diesel')]
choices = [0, 1]
choice_data_subset['Fuelchoice'] = np.select(conditions, choices, default=2)
choice_data_subset

C:\Users\Lasse\AppData\Local\Temp\ipykernel_18776\762087676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset['Fuelchoice'] = np.select(conditions, choices, default=2)


ID    Fuel  Benzin  Diesel  El                   Make-model-year-fuel  \
0        0  Benzin       1       0   0             toyota-avensis-2006-Benzin   
1        1  Diesel       0       1   0          volkswagen-passat-2006-Diesel   
2        2  Benzin       1       0   0               renault-clio-2006-Benzin   
3        3  Benzin       1       0   0               suzuki-swift-2006-Benzin   
4        4  Benzin       1       0   0                 fiat-punto-2006-Benzin   
...    ...     ...     ...     ...  ..                                    ...   
3353  2785  Diesel       0       1   0               dacia-duster-2020-Diesel   
3378  2786  Benzin       1       0   0  volkswagen-golf-sportsvan-2020-Benzin   
3379  2787      El       0       0   1                  volkswagen-up-2020-El   
3380  2788  Benzin       1       0   0         volkswagen-t-cross-2020-Benzin   
3381  2789  Benzin       1       0   0           volkswagen-t-roc-2020-Benzin   

      Year        Make    market_year    MarketShare  Weight (kg)  \
0     2006      toyota 0.033475590202 0.001802670663         1820   
1     2006  volkswagen 0.033167229938 0.001786065369         2170   
2     2006     renault 0.027863433407 0.001500454321         1425   
3     2006      suzuki 0.027172706416 0.001463258463         1475   
4     2006        fiat 0.023083849323 0.001243072272         1590   
...    ...         ...            ...            ...          ...   
3353  2020       dacia 0.000386870922 0.000027232681         1848   
3378  2020  volkswagen 0.003859273343 0.000271662602         1930   
3379  2020  volkswagen 0.001745637087 0.000122879172         1530   
3380  2020  volkswagen 0.014007558137 0.000986022328         1760   
3381  2020  volkswagen 0.015564477701 0.001095617265         1860   

      Engine effect (kW)   Prices (2015-DKK)  Cost/km (DKK)       Size (m3)  \
0       108.000000000000 403349.299100000004 0.813669065000 12.573440000000   
1       103.000000000000 378504.672900000005 0.681879195000 12.734904000000   
2        43.000000000000 134344.626199999999 0.677245509000  8.872728000000   
3        68.000000000000 163536.214999999997 0.734415584000  9.379500000000   
4        48.000000000000 150287.383200000011 0.689634146000  9.483248000000   
...                  ...                 ...            ...             ...   
3353     85.000000000000 170224.371400000004 0.401119403000 13.202280000000   
3378    110.000000000000 303593.810400000017 0.583653846000 12.411532000000   
3379     61.000000000000 153689.555099999998 0.306765224000  8.855775289000   
3380     85.000000000000 263941.972899999993 0.541964286000 11.429088000000   
3381    110.000000000000 254948.742700000003 0.583653846000 12.086802000000   

           Horsepower  Fuelchoice  
0    139.307692300000           2  
1    128.333333300000           1  
2     60.000000000000           2  
3     94.000000000000           2  
4     63.888888890000           2  
...               ...         ...  
3353 103.750000000000           1  
3378 150.000000000000           2  
3379  61.222222220000           0  
3380 115.000000000000           2  
3381 150.000000000000           2  

[2790 rows x 17 columns]

In [27]:
choice_data_subset['market_year'] = choice_data_subset['market_year'].astype(str).astype(float)

C:\Users\Lasse\AppData\Local\Temp\ipykernel_18776\1687196643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset['market_year'] = choice_data_subset['market_year'].astype(str).astype(float)


In [115]:
choice_data_subset.rename(columns = {'Make-model-year-fuel': 'Key'}, inplace = True)
choice_data_subset.head()
choice_data_subset.to_csv(path + 'clogit__data_subset.csv')

C:\Users\Lasse\AppData\Local\Temp\ipykernel_18776\3740217262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset.rename(columns = {'Make-model-year-fuel': 'Key'}, inplace = True)


In [14]:
#choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 100
#choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100
#choice_data_subset['Prices (2015-DKK)'] = choice_data_subset['Prices (2015-DKK)'] / 1000
#choice_data_subset['Horsepower'] = choice_data_subset['Horsepower'] / 100
#choice_data_subset['Size (m3)'] = choice_data_subset['Size (m3)'] / 10
#choice_data_subset['Cost/km (DKK)'] = choice_data_subset['Cost/km (DKK)'] / 10
#choice_data_subset

In [15]:
N = choice_data_subset.Year.nunique()
N = int(N)
N

15

In [16]:
#J = choice_data_subset.(Year+Make).nunique()
J = choice_data_subset.Key.nunique()/N
J = int(J)
#J = choice_data_subset.Fuel.nunique()
J

186

In [56]:
# x_vars = list(choice_data_subset.loc[7:12].index.values)
x_vars = list(choice_data_subset.iloc[:,[9,10,11,13,14]])
x_vars

['Weight (kg)',
 'Engine effect (kW)',
 'Prices (2015-DKK)',
 'Size (m3)',
 'Horsepower']

In [57]:
x_lab = ['Weight (kg)', 'Engine effect (kW)', 'Prices (2015-DKK)', 'Cost/km (DKK)', 'Horsepower']

In [58]:
K = len(x_vars)
K

5

In [120]:
y = choice_data_subset['MarketShare'].values.reshape(N,J)

In [121]:
# y = y.astype(str).astype(float)
y

array([[1.80267066e-03, 1.78606537e-03, 1.50045432e-03, ...,
        2.19189874e-05, 3.28784811e-05, 1.69373994e-05],
       [2.39448332e-04, 1.84584443e-03, 1.46126583e-03, ...,
        4.91516688e-05, 2.13876180e-04, 3.08858459e-05],
       [4.64948218e-05, 3.32105870e-05, 2.43765709e-04, ...,
        2.82289990e-05, 7.60522442e-05, 2.19189874e-05],
       ...,
       [3.51700116e-04, 3.19485847e-04, 7.50891372e-04, ...,
        7.09378138e-04, 1.92621405e-05, 2.15868816e-05],
       [2.36459379e-04, 2.29153050e-05, 9.25911166e-04, ...,
        1.59410818e-05, 3.98527044e-05, 1.77012429e-04],
       [3.78932798e-04, 1.76016111e-05, 2.65352590e-04, ...,
        1.22879172e-04, 9.86022328e-04, 1.09561727e-03]])

In [122]:
x = choice_data_subset[x_vars].values.reshape((N,J,K))
x.shape

(15, 186, 5)

In [123]:
theta = 0.0 * np.ones((K,))
(x @ theta).shape

(15, 186)

In [124]:
theta0 = np.zeros((K,)) 
theta0

array([0., 0., 0., 0., 0.])

In [125]:
#choice_data_subset.fillna(0)
result = est2.estimate(logit.q, theta0, y, x)
result

         Current function value: 0.343005
         Iterations: 18
         Function evaluations: 384
         Gradient evaluations: 62


{'theta_hat': array([-2.59154789e-04, -2.94340263e-04, -1.67506183e-07, -1.28412789e-01,
        -7.50854662e-04]),
 'success': False,
 'nit': 18,
 'nfev': 384,
 'fun': 0.3430051262164039,
 'se': array([5.74865453e-01, 3.62964536e+00, 7.32371985e-04, 1.09318293e+02,
        1.76989096e+00]),
 't': array([-4.50809468e-04, -8.10933943e-05, -2.28717354e-04, -1.17466881e-03,
        -4.24237810e-04]),
 'cov': array([[ 3.30470289e-01,  1.10255819e+00, -1.94257226e-04,
         -6.01580646e+01,  2.46796885e-01],
        [ 1.10255819e+00,  1.31743255e+01, -1.73828738e-03,
         -2.55421005e+02,  1.40680898e+00],
        [-1.94257226e-04, -1.73828738e-03,  5.36368725e-07,
          4.60361679e-02, -7.74389077e-04],
        [-6.01580646e+01, -2.55421005e+02,  4.60361679e-02,
          1.19504892e+04, -7.68294492e+01],
        [ 2.46796885e-01,  1.40680898e+00, -7.74389077e-04,
         -7.68294492e+01,  3.13251400e+00]])}

In [126]:
#result['theta']

In [127]:
est2.print_table(x_lab, result)

Optimizer failed after 18 iter. (384 func. evals.). Final criterion:    0.343.
Results


theta               se               t
Weight (kg)        -0.000300000000   0.574900000000 -0.000500000000
Engine effect (kW) -0.000300000000   3.629600000000 -0.000100000000
Prices (2015-DKK)  -0.000000000000   0.000700000000 -0.000200000000
Cost/km (DKK)      -0.128400000000 109.318300000000 -0.001200000000
Horsepower         -0.000800000000   1.769900000000 -0.000400000000

In [108]:
y = choice_data_subset['Fuelchoice'].values.reshape(N,J)

In [109]:
y = y[:, 0] # all J elements are identical along axis=1

In [110]:
y

array([2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0])

In [113]:
res = est.estimate(clogit.q, theta0, y, x)

         Current function value: 4.866396
         Iterations: 9
         Function evaluations: 354
         Gradient evaluations: 57


In [114]:
cho = pd.DataFrame({v:res[v] for v in ['theta', 'se', 't']}, index=x_lab)
cho

theta             se               t
Weight (kg)         0.003670199143 0.002510180726  1.462125457918
Engine effect (kW) -0.002760793090 0.004549419782 -0.606845097322
Prices (2015-DKK)   0.000000064693 0.000002643037  0.024476864020
Cost/km (DKK)      -0.659308144769 0.382664290436 -1.722941390788
Horsepower          0.003484725941 0.010048475767  0.346791495736

In [ ]:
choice_subset_long = pd.melt(choice_data_subset, id_vars='Year', value_vars=['Fuel', 'Benzin', 'Diesel', 'El', 'Make', 'Weight (kg)', 'Engine effect (kW)', 'Prices (2015-DKK)', 'Cost/km (DKK)', 'Size (m3)', 'Horsepower'])

In [ ]:
choice_subset_long

In [ ]:
#indexes_to_drop = []
y = 2006
for x in range(2006,2021):
    #print(len(choice_data.loc[choice_data['Year'] == y])) 
    start = len(choice_data.loc[choice_data['Year'] == 2006 ])
    ny = len(choice_data.loc[choice_data['Year'] == y ])
    #diff = ny - start
    #print(diff)
    while start < ny:
            m = choice_data.loc[(choice_data['Year'] == y) & (choice_data['El'] != 1), 'Registrations'].idxmin()
            choice_data.drop(index=m,inplace=True)
            #for i in choice_data[choice_data['Year'] == y].index:
            #print(choice_data['Registrations'].loc[i])
            #choice_data.drop(choice_data.nsmallest(diff, 'Registrations'.index)
            #if choice_data['Registrations'].loc[i] < choice_data['Registrations'].loc[i+1]:
                #indexes_to_drop.append(i)
            #choice_data.drop(choice_data.index[indexes_to_drop], inplace=True)
            #i += 1
    y += 1

In [ ]:
final = pd.read_csv(path + 'finales.csv', delimiter=',', encoding = 'unicode_escape')

In [ ]:
final

In [ ]:
y = 2006
for x in range(2006,2021):
    start = len(final.loc[final['Year'] == 2006 ])
    ny = len(final.loc[final['Year'] == y ])
    while start < ny:
            m = final.loc[(final['Year'] == y) & (final['El'] != 1), 'Shares'].idxmin()
            final.drop(index=m,inplace=True)
    y += 1